# .Net Core Data Core Client Demo
This notebook will demonstrate how to use the Intelligent Plant Data Core client in the dotnet interactive Jupyter kernel.

## Installing

Cell magic can be used to install the data core client from the standard nuget package source.

In [1]:
#r "nuget:IntelligentPlant.DataCore.HttpClient,1.0.0-alpha29"
using IntelligentPlant.DataCore.Client;
using IntelligentPlant.DataCore.Client.Clients;

Installed package IntelligentPlant.DataCore.HttpClient version 1.0.0-alpha29

This cell contains the boiler plate code needed to set up the data core client using the Jupyter server's preconfigured access token.

In [2]:
using System.Net.Http;
using System.Net.Http.Headers;
using System.Threading;
var accessToken = System.Environment.GetEnvironmentVariable("ACCESS_TOKEN", EnvironmentVariableTarget.Process);
var authenticationHandler = DataCoreHttpClient.CreateAuthenticationMessageHandler(async (HttpRequestMessage request, object context, CancellationToken ct) => {
    return new AuthenticationHeaderValue("Bearer", accessToken);
});
var primaryHandler = new SocketsHttpHandler();
authenticationHandler.InnerHandler = primaryHandler;
var baseClient = new HttpClient(authenticationHandler);
var dataCore = new DataSourcesClient<object, DataCoreHttpClientOptions>(baseClient, new DataCoreHttpClientOptions() {
    DataCoreUrl = new System.Uri("https://appstore.intelligentplant.com/gestalt/datacore/")
});

The data core client is now configured.

The client can be used to get a list of available data sources (that have been authorised for the Jupyter Hub app).

In [3]:
var datasourceResp = await dataCore.GetDataSourcesAsync();

An expression without a ';' will be displayed as the cell output.

In [4]:
datasourceResp.Select(x => x.Name.QualifiedName)

index,value
0,IP Datasource
1,IP Datasource 2
2,Vibration


The client can be used to query for available tags.

In this case we'll get the 1st 5 tags from the demo data source "IP Data source 2".

(Make sure this data source appears in the list of authorised data sources)

In [5]:
var dsn = "IP Datasource 2";

In [6]:
var resp = await dataCore.FindTagsAsync(dsn, nameFilter: "*", page: 1, pageSize: 5);

In [7]:
var tags = resp.Select(x => x.Name).ToArray();

In [8]:
tags

index,value
0,Air Scour Blowers_Outlet_PI
1,Antifoam Inj Pump A_PI
2,Antifoam Inj Pump B_PI
3,Anti-Scalent Pump A_PI
4,Anti-Scalent Pump B_PI


To query a data source for tag data a tag map must be created.

The map should have qualified data source names as keys and arrays of tag names as values.

This tag map is setup to query "IP Datasource 2" for the 1st 5 tags that we requested from it earlier.

In [9]:
var tagMap = new Dictionary<string, string[]>();
tagMap[dsn] = tags;

To query for plot data use the `ReadPlotTagValuesAsync(..)` function. This function takes a tag map, query start time, query end time and number of samples as parameters.

In [10]:
var dataResp = await dataCore.ReadPlotTagValuesAsync(tagMap, "*-30d", "*", 100);

The response is a dictionary where the keys are the data source names are the keys.

The value for each data source is a nested dictionary where the keys are the requested tag names and the values are a historical tag value object, which contains an array of tag values.

In [11]:
dataResp.Keys

index,value
0,IP Datasource 2


In [12]:
dataResp[dsn].Keys

index,value
0,Antifoam Inj Pump A_PI
1,Antifoam Inj Pump B_PI
2,Air Scour Blowers_Outlet_PI
3,Anti-Scalent Pump A_PI
4,Anti-Scalent Pump B_PI


In [13]:
dataResp[dsn][tags.First()]

TagName DisplayType Values Air Scour Blowers_Outlet_PI TrailingEdge [ { IntelligentPlant.DataCore.Client.Model.TagValue: TagName: Air Scour Blowers_Outlet_PI, UtcSampleTime: 2020-06-15 13:11:09Z, NumericValue: 0.00674237338672134, TextValue: 0.00674237338672134, Status: Good, Unit: , Notes: <null>, Error: <null>, IsNumeric: True, HasError: False }, { IntelligentPlant.DataCore.Client.Model.TagValue: TagName: Air Scour Blowers_Outlet_PI, UtcSampleTime: 2020-06-15 20:27:31Z, NumericValue: 2.23927533717533E-05, TextValue: 2.23927533717533E-05, Status: Good, Unit: , Notes: <null>, Error: <null>, IsNumeric: True, HasError: False }, { IntelligentPlant.DataCore.Client.Model.TagValue: TagName: Air Scour Blowers_Outlet_PI, UtcSampleTime: 2020-06-16 03:43:52Z, NumericValue: 0.000311716576711327, TextValue: 0.000311716576711327, Status: Good, Unit: , Notes: <null>, Error: <null>, IsNumeric: True, HasError: False }, { IntelligentPlant.DataCore.Client.Model.TagValue: TagName: Air Scour Blowers_Outlet_PI, UtcSampleTime: 2020-06-16 11:00:14Z, NumericValue: 0.000386630664301369, TextValue: 0.000386630664301369, Status: Good, Unit: , Notes: <null>, Error: <null>, IsNumeric: True, HasError: False }, { IntelligentPlant.DataCore.Client.Model.TagValue: TagName: Air Scour Blowers_Outlet_PI, UtcSampleTime: 2020-06-16 18:16:36Z, NumericValue: 0.000471998362192066, TextValue: 0.000471998362192066, Status: Good, Unit: , Notes: <null>, Error: <null>, IsNumeric: True, HasError: False }, { IntelligentPlant.DataCore.Client.Model.TagValue: TagName: Air Scour Blowers_Outlet_PI, UtcSampleTime: 2020-06-17 01:32:58Z, NumericValue: 0.201384109379807, TextValue: 0.201384109379807, Status: Good, Unit: , Notes: <null>, Error: <null>, IsNumeric: True, HasError: False }, { IntelligentPlant.DataCore.Client.Model.TagValue: TagName: Air Scour Blowers_Outlet_PI, UtcSampleTime: 2020-06-17 08:49:20Z, NumericValue: 0.000312429592030405, TextValue: 0.000312429592030405, Status: Good, Unit: , Notes: <null>, Error: <null>, IsNumeric: True, HasError: False }, { IntelligentPlant.DataCore.Client.Model.TagValue: TagName: Air Scour Blowers_Outlet_PI, UtcSampleTime: 2020-06-17 16:05:41Z, NumericValue: 0.000411417831222486, TextValue: 0.000411417831222486, Status: Good, Unit: , Notes: <null>, Error: <null>, IsNumeric: True, HasError: False }, { IntelligentPlant.DataCore.Client.Model.TagValue: TagName: Air Scour Blowers_Outlet_PI, UtcSampleTime: 2020-06-17 23:22:03Z, NumericValue: 0.000486377189824725, TextValue: 0.000486377189824725, Status: Good, Unit: , Notes: <null>, Error: <null>, IsNumeric: True, HasError: False }, { IntelligentPlant.DataCore.Client.Model.TagValue: TagName: Air Scour Blowers_Outlet_PI, UtcSampleTime: 2020-06-18 06:38:25Z, NumericValue: 0.000195162860541375, TextValue: 0.000195162860541375, Status: Good, Unit: , Notes: <null>, Error: <null>, IsNumeric: True, HasError: False }, { IntelligentPlant.DataCore.Client.Model.TagValue: TagName: Air Scour Blowers_Outlet_PI, UtcSampleTime: 2020-06-18 13:54:47Z, NumericValue: 0.0034443941672715, TextValue: 0.0034443941672715, Status: Good, Unit: , Notes: <null>, Error: <null>, IsNumeric: True, HasError: False }, { IntelligentPlant.DataCore.Client.Model.TagValue: TagName: Air Scour Blowers_Outlet_PI, UtcSampleTime: 2020-06-18 21:11:09Z, NumericValue: 0.000952211607680374, TextValue: 0.000952211607680374, Status: Good, Unit: , Notes: <null>, Error: <null>, IsNumeric: True, HasError: False }, { IntelligentPlant.DataCore.Client.Model.TagValue: TagName: Air Scour Blowers_Outlet_PI, UtcSampleTime: 2020-06-19 04:27:31Z, NumericValue: 0.0003143188945733, TextValue: 0.0003143188945733, Status: Good, Unit: , Notes: <null>, Error: <null>, IsNumeric: True, HasError: False }, { IntelligentPlant.DataCore.Client.Model.TagValue: TagName: Air Scour Blowers_Outlet_PI, UtcSampleTime: 2020-06-19 11:43:52Z, NumericValue: 0.000194766211514327, TextValue: 0.000194766211514327, Status: Good, Unit: , Notes: <null>, Error: <null>, IsNumeric: True, Has

The each tag value is an object containg meta data and the tag value.

Depending on the data function the sample times may not be at consistent intervals and therefore may differ from 1 tag to another.

In [14]:
dataResp[dsn][tags.First()].Values.First()

TagName,UtcSampleTime,NumericValue,TextValue,Status,Unit,Notes,Error,IsNumeric,HasError
Air Scour Blowers_Outlet_PI,2020-06-15 13:11:09Z,0.00674237338672134,0.00674237338672134,Good,,<null>,<null>,True,False


To extract the numeric data for a given tag use Linq

In [15]:
dataResp[dsn][tags.First()].Values.Select(val => val.NumericValue)

index,value
0,0.00674237338672134
1,2.23927533717533E-05
2,0.000311716576711327
3,0.000386630664301369
4,0.000471998362192066
5,0.201384109379807
6,0.000312429592030405
7,0.000411417831222486
8,0.000486377189824725
9,0.000195162860541375


The XPlot Library lets us easily plot the tag data.

In [16]:
using XPlot.Plotly;

In [17]:
var tagSeries = new List<Graph.Scatter>();

foreach (var tag in tags) {
    
    var data = dataResp[dsn][tag].Values.Select(x => x.NumericValue);
    
    var min = data.Min();
    
    var max = data.Max() - min;
    
    var series = new Graph.Scatter
    {
        name = tag,
        x = dataResp[dsn][tag].Values.Select(x => x.UtcSampleTime),
        y = data.Select(x => (x - min)/max) //normalise the data into the range 0-1
    };
    
    tagSeries.Add(series);
}

var chart = Chart.Plot(tagSeries);
display(chart);

In [18]:
var openSeries = new Graph.Scatter
{
    mode = "markers",
    x = dataResp[dsn]["Anti-Scalent Pump A_PI"].Values.Select(x => x.NumericValue),
    y = dataResp[dsn]["Anti-Scalent Pump B_PI"].Values.Select(x => x.NumericValue)
};


var chart = Chart.Plot(new[] {openSeries});
chart.WithTitle("Anti-Scalent Pump A_PI vs Anti-Scalent Pump B_PI");
display(chart);